# Importing Neccesary Libraries

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

# Loading Pickled file of the Best CNN Model 

In [2]:
with open('model_pickle4','rb')as f:
    model4=pickle.load(f)

# Function for Converting PDF to Images

In [3]:
import PyPDF2
import fitz
import pytesseract
from PIL import Image


def extract_image_from_pdf(pdf_file):
    pdf = fitz.open(pdf_file)

    for page_number in range(len(pdf)):
        page = pdf[page_number]
        image_list = page.get_images(full=True)

        for image_index, img in enumerate(image_list):
            xref = img[0]
            base_image = pdf.extract_image(xref)
            image_data = base_image["image"]

            with open(f"image_page_{page_number + 1}_index_{image_index + 1}.png", "wb") as image_file:
                image_file.write(image_data)


pdf_file = 'MANGILAL.pdf'  
extract_image_from_pdf(pdf_file)

# Function for converting Orientation of the Images by rotating it according to model output

In [4]:
from PIL import Image
def rotate_images(dir_path, model):
    for imgs in os.listdir(dir_path):
        img = image.load_img(dir_path+'//'+imgs ,target_size=(64,64))
        img_array =image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        y_hat = model4.predict(img_array)
        y_pred = np.argmax(y_hat, axis=1)[0]
        y=int(y_pred)
        img_pr = (dir_path+'//'+imgs)

        if y==0:
            new_img=Image.open(img_pr)
            new_img = new_img.rotate(angle=180, expand=True)
            new_img.save(f"result/rotated_{imgs}")
        elif y==1:
            new_img=Image.open(img_pr)
            new_img = new_img.rotate(angle=270, expand=True)
            new_img.save(f"result/rotated_{imgs}")
        elif y==2:
            new_img=Image.open(img_pr)
            new_img = new_img.rotate(angle=90, expand=True)
            new_img.save(f"result/rotated_{imgs}")
        else:
            new_img=Image.open(img_pr)
            new_img=new_img.rotate(angle=0,expand=True)
            new_img.save(f"result/rotated_{imgs}")
dir_path='test'
model=model4
rotate_images(dir_path, model)

1/1 [==============================] - 0s 33ms/step


# Function for Compilation of the Images to Pdf

In [ ]:
from fpdf import FPDF
def convert_images_to_pdf(dir_path):
    pdf = FPDF()
    image_paths=os.listdir(dir_path)
    for image_path in image_paths:
        pdf.add_page()
        pdf.image(dir_path+'/'+image_path, 0, 0, 210, 297)

    pdf.output("output_def.pdf")

    
dir_path='result'
convert_images_to_pdf(dir_path)